# Análisis de Atribución aplicado a marketing

## Librerías

In [1]:
import pandas as pd
import numpy as np
import os

# Funciones

Se crearán los siguientes parámetros:
- df: DataFrame con los datos de interacción de las campañas.
- user_id_col: Nombre de la columna que contiene los identificadores de usuario.
- campaign_source_col: Nombre de la columna que contiene los nombres de las campañas.
- timestamp_col: Nombre de la columna que contiene las marcas de tiempo de las interacciones.
- conversion_col: Nombre de la columna que indica si hubo conversión (1) o no (0).

1. Atribución al último click

In [2]:
def atribucion_ultimo_click(df, user_id_col, campaign_source_col, timestamp_col, conversion_col):

    # Ordenamos por usuario y las fechas de interacción
    df = df.sort_values(by=[user_id_col, timestamp_col])
    
    # Obtenemos la última campaña
    last_click_attribution = df[df[conversion_col] == 1].groupby(user_id_col).last().reset_index()
    
    # Resumen de conversiones por campaña
    conversion_summary = last_click_attribution[campaign_source_col].value_counts().reset_index()
    conversion_summary.columns = [campaign_source_col, 'CONVERSIONES']
    
    return conversion_summary

2. Atribución lineal

In [3]:
def atribucion_lineal (df, user_id_col, campaign_source_col, timestamp_col, conversion_col):
    
    # Ordenamos por usuario y timestamp
    df = df.sort_values(by=[user_id_col, timestamp_col])
    
    # Creamos una columna de contribución inicializada en 0
    df['CONTRIBUCION'] = 0.0
    
    # Obtenemos los usuarios que tuvieron una conversión
    users_with_conversions = df[df[conversion_col] == 1][user_id_col].unique()
    
    # Iteramos sobre cada usuario que tuvo una conversión
    for user_id in users_with_conversions:
        # Obtenemos las interacciones de este usuario
        user_interactions = df[df[user_id_col] == user_id]
        
        # Obtenemos el timestamp de la conversión
        conversion_time = user_interactions[user_interactions[conversion_col] == 1][timestamp_col].max()
        
        # Filtramos las interacciones antes de la conversión
        interactions_before_conversion = user_interactions[user_interactions[timestamp_col] <= conversion_time]
        
        # Calculamos la contribución uniforme para cada interacción antes de la conversión
        contribution_value = 1.0 / len(interactions_before_conversion)
        df.loc[interactions_before_conversion.index, 'CONTRIBUCION'] = contribution_value
    
    # Resumen de contribuciones por campaña
    attribution_summary = df.groupby(campaign_source_col)['CONTRIBUCION'].sum().reset_index()
    attribution_summary.columns = [campaign_source_col, 'CONVERSIONES']
    
    return attribution_summary

3. Atribución en U

In [4]:
def atribucion_u_shape (df, user_id_col, campaign_source_col, timestamp_col, conversion_col):
    
    # Ordenamos por usuario y timestamp
    df = df.sort_values(by=[user_id_col, timestamp_col])
    
    # Creamos una columna de contribución inicializada en 0
    df['CONTRIBUCION'] = 0.0
    
    # Obtenemos los usuarios que tuvieron una conversión
    users_with_conversions = df[df[conversion_col] == 1][user_id_col].unique()
    
    # Iteramos sobre cada usuario que tuvo una conversión
    for user_id in users_with_conversions:
        # Obtenemos las interacciones de este usuario
        user_interactions = df[df[user_id_col] == user_id]
        
        # Obtenemos el timestamp de la conversión
        conversion_time = user_interactions[user_interactions[conversion_col] == 1][timestamp_col].max()
        
        # Filtramos las interacciones antes de la conversión
        interactions_before_conversion = user_interactions[user_interactions[timestamp_col] <= conversion_time]
        
        num_interactions = len(interactions_before_conversion)
        
        if num_interactions == 1:
            # Si solo hay un punto de contacto, se le asigna todo el crédito
            df.loc[interactions_before_conversion.index, 'CONTRIBUCION'] = 1.0
        elif num_interactions == 2:
            # Si hay dos puntos de contacto, se distribuye el crédito al 50%
            df.loc[interactions_before_conversion.index, 'CONTRIBUCION'] = 0.5
        else:
            # Asignar 40% al primer y último punto de contacto
            first_interaction_idx = interactions_before_conversion.index[0]
            last_interaction_idx = interactions_before_conversion.index[-1]
            df.loc[first_interaction_idx, 'CONTRIBUCION'] = 0.4
            df.loc[last_interaction_idx, 'CONTRIBUCION'] = 0.4
            
            # Asignar el 20% restante de manera uniforme a los puntos de contacto intermedios
            intermediate_interactions = interactions_before_conversion.iloc[1:-1]
            intermediate_contribution = 0.2 / len(intermediate_interactions)
            df.loc[intermediate_interactions.index, 'CONTRIBUCION'] = intermediate_contribution
    
    # Resumen de contribuciones por campaña
    attribution_summary = df.groupby(campaign_source_col)['CONTRIBUCION'].sum().reset_index()
    attribution_summary.columns = [campaign_source_col, 'CONVERSIONES']
    
    return attribution_summary

4. Atribución basada en decaimiento temporal

In [5]:
def time_decay_attribution (df, user_id_col, campaign_source_col, timestamp_col, conversion_col, half_life):
    
    # Ordenamos por usuario y timestamp
    df = df.sort_values(by=[user_id_col, timestamp_col])
    
    # Creamos una columna de contribución inicializada en 0
    df['CONTRIBUCION'] = 0.0
    
    # Obtenemos los usuarios que tuvieron una conversión
    users_with_conversions = df[df[conversion_col] == 1][user_id_col].unique()
    
    # Iteramos sobre cada usuario que tuvo una conversión
    for user_id in users_with_conversions:
        # Obtenemos las interacciones de este usuario
        user_interactions = df[df[user_id_col] == user_id]
        
        # Obtenemos el timestamp de la conversión
        conversion_time = user_interactions[user_interactions[conversion_col] == 1][timestamp_col].max()
        
        # Filtramos las interacciones antes de la conversión
        interactions_before_conversion = user_interactions[user_interactions[timestamp_col] <= conversion_time]
        
        # Calcular el tiempo en días desde cada interacción hasta la conversión
        time_diff = (conversion_time - interactions_before_conversion[timestamp_col]).dt.total_seconds() / (60 * 60 * 24)
        
        # Calcular el decaimiento exponencial
        decay_factors = np.exp(-time_diff / half_life)
        
        # Normalizar los factores de decaimiento para que sumen a 1
        decay_factors /= decay_factors.sum()
        
        # Asignar los valores de contribución
        df.loc[interactions_before_conversion.index, 'CONTRIBUCION'] = decay_factors
    
    # Resumen de contribuciones por campaña
    attribution_summary = df.groupby(campaign_source_col)['CONTRIBUCION'].sum().reset_index()
    attribution_summary.columns = [campaign_source_col, 'CONVERSIONES']
    
    return attribution_summary

# Fuente de datos

In [6]:
mkt_df=pd.read_excel('Fuente_Datos_Recreados.xlsx')

In [7]:
mkt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               51 non-null     int64         
 1   USER_ID          51 non-null     int64         
 2   CAMPAIGN_SOURCE  51 non-null     object        
 3   FECHA            51 non-null     datetime64[ns]
 4   CONVERSION       51 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 2.1+ KB


In [8]:
#Ordenamos el dataframe por usuario y por fecha de impacto en marketing
mkt_df=mkt_df.sort_values(by=['USER_ID', 'FECHA'])

In [9]:
mkt_df[mkt_df['CONVERSION'] == 1]

,ID,USER_ID,CAMPAIGN_SOURCE,FECHA,CONVERSION
9,10,28237077,EMAIL,2024-07-01,1
50,51,28237077,SEO,2024-07-30,1
22,23,41875735,WHATSAPP,2024-07-20,1
34,35,41875735,SOCIAL MEDIA,2024-07-25,1
41,42,42322256,ADS,2024-07-27,1
23,24,46642299,WHATSAPP,2024-07-20,1
35,36,46642299,SOCIAL MEDIA,2024-07-25,1
6,7,75818490,EMAIL,2024-07-01,1
46,47,75818490,WHATSAPP,2024-07-27,1
2,3,75860391,EMAIL,2024-07-01,1


# Análisis

1. Atribución al último click

In [10]:
conversion_uc=atribucion_ultimo_click(mkt_df,'USER_ID','CAMPAIGN_SOURCE','FECHA','CONVERSION')

In [11]:
conversion_uc

,CAMPAIGN_SOURCE,CONVERSIONES
0,SOCIAL MEDIA,2
1,ADS,2
2,SEO,1
3,WHATSAPP,1


2. Atribución lineal

In [12]:
conversion_lineal = atribucion_lineal (mkt_df, 'USER_ID','CAMPAIGN_SOURCE','FECHA','CONVERSION')

In [13]:
conversion_lineal

,CAMPAIGN_SOURCE,CONVERSIONES
0,ADS,1.000000
1,EMAIL,1.833333
2,SEO,0.333333
3,SOCIAL MEDIA,1.000000
4,WHATSAPP,1.833333


3. Atribución en U

In [14]:
conversion_u_shape = atribucion_u_shape (mkt_df, 'USER_ID','CAMPAIGN_SOURCE','FECHA','CONVERSION')

In [15]:
conversion_u_shape

,CAMPAIGN_SOURCE,CONVERSIONES
0,ADS,1.0
1,EMAIL,1.9
2,SEO,0.4
3,SOCIAL MEDIA,1.0
4,WHATSAPP,1.7


4. Atribución basada en decaimiento temporal

In [16]:
# Definimos una vida media (half-life) de 7 días para el decaimiento exponencial
half_life = 7

In [17]:
conversion_summary = time_decay_attribution (mkt_df, 'USER_ID','CAMPAIGN_SOURCE','FECHA','CONVERSION', half_life)

In [18]:
conversion_summary

,CAMPAIGN_SOURCE,CONVERSIONES
0,ADS,1.952414
1,EMAIL,0.080902
2,SEO,0.599766
3,SOCIAL MEDIA,1.342695
4,WHATSAPP,2.024223
